In [106]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from PythonScripts import wrangle

In [2]:
df = wrangle.get_curriculum_log()

In [3]:
df.isna().sum()

endpoint        0
user_id         0
cohort_id       0
source_ip       0
name            0
start_date      0
end_date        0
created_at      0
updated_at      0
program_id      0
program_name    0
dtype: int64

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 954312 entries, 2018-01-26 to 2021-07-15
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   endpoint      954312 non-null  object        
 1   user_id       954312 non-null  int64         
 2   cohort_id     954312 non-null  float64       
 3   source_ip     954312 non-null  object        
 4   name          954312 non-null  object        
 5   start_date    954312 non-null  datetime64[ns]
 6   end_date      954312 non-null  datetime64[ns]
 7   created_at    954312 non-null  datetime64[ns]
 8   updated_at    954312 non-null  datetime64[ns]
 9   program_id    954312 non-null  int64         
 10  program_name  954312 non-null  object        
dtypes: datetime64[ns](4), float64(1), int64(2), object(4)
memory usage: 87.4+ MB


In [5]:
#6 what topics are grads continuing to access after graduation? and into there jobs for each program
df.loc[(df.program_name == 'Data Science') & (df.updated_at > df.end_date)].value_counts().head(10)

Series([], dtype: int64)

In [6]:
df.loc[(df.program_name == 'PHP') & (df.updated_at > df.end_date)].endpoint.value_counts()[1:].head(20)

javascript-i                                                                 759
html-css                                                                     709
javascript-i/javascript-with-html                                            709
javascript-i/introduction/working-with-data-types-operators-and-variables    485
html-css/css-ii/bootstrap-grid-system                                        387
javascript-i/introduction/operators                                          387
jquery                                                                       385
javascript-i/functions                                                       379
javascript-ii                                                                348
toc                                                                          340
javascript-i/loops                                                           323
html-css/css-ii/bootstrap-introduction                                       307
javascript-i/conditionals   

In [7]:
df.loc[(df.program_name == 'Java') & (df.updated_at > df.end_date)].endpoint.value_counts()[2:].head(20)

javascript-i                                2792
spring                                      2289
html-css                                    2247
java-iii                                    2219
java-ii                                     2093
toc                                         1934
appendix                                    1827
javascript-ii                               1599
java-i                                      1577
jquery                                      1564
mysql                                       1495
jquery/ajax/weather-map                     1292
classification/overview                     1160
sql/mysql-overview                          1066
classification/scale_features_or_not.svg    1028
spring/fundamentals/repositories            1025
java-iii/servlets                            965
spring/fundamentals/views                    899
java-iii/jsp-and-jstl                        839
mysql/tables                                 809
Name: endpoint, dtyp

In [8]:
df.loc[(df.program_name == 'Front End') & (df.updated_at > df.end_date)].endpoint.value_counts().head(10)

content/html-css                               2
content/html-css/gitbook/images/favicon.ico    1
content/html-css/introduction.html             1
/                                              1
Name: endpoint, dtype: int64

In [9]:
#5 at some point in 2019 the ability for students to access both curriculums was deprecated, do you see evidence of that happening, did it happen before?

##classes in 2020 and greater can only access there specific stuff 
java_list = df[(df['program_name'] == 'Java') & (df.created_at.dt.year > 2019)].endpoint.unique().tolist()

ds_list = df[(df['program_name'] == 'Data Science') & (df.created_at.dt.year > 2019)].endpoint.unique().tolist()

#look at common pages visited for both these cohorts to filter by
common_list = []
for j in java_list:
    for d in ds_list:
        if j == d:
            common_list.append(j)

In [10]:
#data science students who accessed the webdev cirrculum before swi
ds_in_web = df.loc[(df['program_name'] == 'Data Science') & (df['endpoint'].isin(java_list)) & (~df['endpoint'].isin(common_list))]

In [11]:
#web dev students who accessed the ds curriculum before switch
web_in_ds = df.loc[(df['program_name'] == 'Java') & (df['endpoint'].isin(ds_list)) & (~df['endpoint'].isin(common_list))]

In [12]:
#data science students who accessed web dev curriculum after the 2019 switch
ds_in_web[ds_in_web['created_at'].dt.year > 2019]

,endpoint,user_id,cohort_id,source_ip,name,start_date,end_date,created_at,updated_at,program_id,program_name
date,,,,,,,,,,,


In [13]:
web_in_ds[web_in_ds['created_at'].dt.year > 2019]

,endpoint,user_id,cohort_id,source_ip,name,start_date,end_date,created_at,updated_at,program_id,program_name
date,,,,,,,,,,,


In [288]:
#Is there a cohort that reffered to a lesson significantly more than other cohorts mayb e glossed over?

In [289]:
#Top 10 Most popular endpoints for java students
java_df = df[(df['program_name'] == 'Java') & (~df['endpoint'].isin(common_list))].endpoint.value_counts().to_frame().reset_index().head(10)

In [290]:
#create a list of those top 10 endpoints
list_of_endpoints = java_df['index'].tolist()
#create a list of cohorts for java students
list_of_cohorts = df[df['program_name'] == 'Java'].name.unique().tolist()

In [327]:
#create a dataframe of the how many times the data was accessed by every cohort individually
java_df = df[(df['name'].isin(list_of_cohorts)) & (df['endpoint'].isin(list_of_endpoints))].groupby(['name','endpoint']).count()
java_df = java_df.reset_index().rename(columns=({'user_id':'counts'}))
java_df = java_df[['name','endpoint','counts']]

In [315]:
#avg page visits for each endpoint
mean_for_java = java_df.groupby('endpoint').counts.mean().to_frame().reset_index().rename(columns=({'counts':'avg'}))
#standard deviation for each endpoint
std_for_java = java_df.groupby('endpoint').counts.std().to_frame().reset_index().rename(columns=({'counts':'stds'}))

In [332]:
#show who visits what end point more than others
for i in list_of_endpoints:
    endpoint_mean = mean_for_java[mean_for_java['endpoint'] == i].avg[mean_for_java[mean_for_java['endpoint'] == i].index[0]]
    endpoint_std = std_for_java[std_for_java['endpoint'] == i].stds[std_for_java[std_for_java['endpoint'] == i].index[0]]
    
    print('Cohorts who accessed the {} lesson more than the other cohorts'.format(i))
    print(java_df[(java_df['endpoint'] == i) & (java_df['counts'] > (endpoint_mean + endpoint_std))].name)
    print('Cohorts who accessed the {} lesson less than the other cohorts'.format(i))
    print(java_df[(java_df['endpoint'] == i) & (java_df['counts'] < (endpoint_mean - endpoint_std))].name)

Cohorts who accessed the javascript-i lesson more than the other cohorts
203    Staff
Name: name, dtype: object
Cohorts who accessed the javascript-i lesson less than the other cohorts
134      Mammoth
163      Niagara
183    Pinnacles
193      Sequoia
Name: name, dtype: object
Cohorts who accessed the java-iii lesson more than the other cohorts
103    Jupiter
202      Staff
Name: name, dtype: object
Cohorts who accessed the java-iii lesson less than the other cohorts
133      Mammoth
162      Niagara
172       Oberon
182    Pinnacles
192      Sequoia
Name: name, dtype: object
Cohorts who accessed the html-css lesson more than the other cohorts
100    Jupiter
199      Staff
Name: name, dtype: object
Cohorts who accessed the html-css lesson less than the other cohorts
130      Mammoth
159      Niagara
179    Pinnacles
189      Sequoia
Name: name, dtype: object
Cohorts who accessed the java-ii lesson more than the other cohorts
102    Jupiter
201      Staff
Name: name, dtype: object
Coho

In [343]:
#Top 10 Most popular endpoints for DS students
ds_df = df[(df['program_name'] == 'Data Science') & (~df['endpoint'].isin(common_list))].endpoint.value_counts().to_frame().reset_index().head(10)

In [344]:
#create a list of those top 10 endpoints
list_of_endpoints = ds_df['index'].tolist()
#create a list of cohorts for java students
list_of_cohorts = df[df['program_name'] == 'Data Science'].name.unique().tolist()

In [345]:
#create a dataframe of the how many times the data was accessed by every cohort individually
ds_df = df[(df['name'].isin(list_of_cohorts)) & (df['endpoint'].isin(list_of_endpoints))].groupby(['name','endpoint']).count()
ds_df = ds_df.reset_index().rename(columns=({'user_id':'counts'}))
ds_df = ds_df[['name','endpoint','counts']]

In [360]:
#avg page visits for each endpoint
mean_for_ds = ds_df.groupby('endpoint').counts.mean().to_frame().reset_index().rename(columns=({'counts':'avg'}))
#standard deviation for each endpoint
std_for_ds = ds_df.groupby('endpoint').counts.std().to_frame().reset_index().rename(columns=({'counts':'stds'}))

In [362]:
#show who visits what end point more than others
for i in list_of_endpoints:
    endpoint_mean = mean_for_ds[mean_for_ds['endpoint'] == i].avg[mean_for_ds[mean_for_ds['endpoint'] == i].index[0]]
    endpoint_std = std_for_ds[std_for_ds['endpoint'] == i].stds[std_for_ds[std_for_ds['endpoint'] == i].index[0]]
    
    print('Cohorts who accessed the {} lesson more than the other cohorts'.format(i))
    print(ds_df[(ds_df['endpoint'] == i) & (ds_df['counts'] > (endpoint_mean + endpoint_std))].name)
    print('Cohorts who accessed the {} lesson less than the other cohorts'.format(i))
    print(ds_df[(ds_df['endpoint'] == i) & (ds_df['counts'] < (endpoint_mean - endpoint_std))].name)

Cohorts who accessed the classification/overview lesson more than the other cohorts
27    Darden
Name: name, dtype: object
Cohorts who accessed the classification/overview lesson less than the other cohorts
7    Bayes
Name: name, dtype: object
Cohorts who accessed the classification/scale_features_or_not.svg lesson more than the other cohorts
28    Darden
Name: name, dtype: object
Cohorts who accessed the classification/scale_features_or_not.svg lesson less than the other cohorts
8    Bayes
Name: name, dtype: object
Cohorts who accessed the 1-fundamentals/modern-data-scientist.jpg lesson more than the other cohorts
2    Bayes
Name: name, dtype: object
Cohorts who accessed the 1-fundamentals/modern-data-scientist.jpg lesson less than the other cohorts
32    Easley
Name: name, dtype: object
Cohorts who accessed the 1-fundamentals/AI-ML-DL-timeline.jpg lesson more than the other cohorts
1    Bayes
Name: name, dtype: object
Cohorts who accessed the 1-fundamentals/AI-ML-DL-timeline.jpg less